# Multidimensional Scaling

### Table of Contents

* Introduction


## Introduction

### About This Notebook

Because of its similarities to PCA, this notebook will not implement multidimensional scaling "by hand," although it would be easy to do by simply replicating the steps in the PCA notebook. Rather, this notebook will utilize the multidimensional scaling functionality built into scikit-learn.

### About This Dataset

This dataset is the NIST handwritten digit classification data set, a data set popular for its simplicity, and for being a nice introduction to image classification for machine learning appications.

### About Multidimensional Scaling

Suppose we have a set of $N$ points, and we know the distance between pairs of points $P_{ij}$ for $i, j = 1 \dots N$. However, we do not know the coordinates of the points or how the distances are calculated. Multidimensional scaling attempts to lower the dimensionality of the data set by reducing the number of dimensions, subject to the restriction that the distances between each of the points should be preserved in the lower dimensional space.

Thus, this is a method focused on preserving the norm of the distance between various points (preserving their relationships), rather than explaining the most amount of variance in the data possible. 

Suppose there is a sample $\mathbf{X}$ with $N$ points, and we're talking about a $d$-dimensional parameter space, $\mathbf{X}_i \in \mathfrak{R}^d$. In our case, the points are observations and the $d$-dimensional space is the pixel data. 

Then the the square of the Euclidean distance between two points $i$ and $j$ is:

$$
d_{ij}^2 = \| \mathbf{X}_i - \mathbf{X}_j \|^2
$$

We construct a matrix from the distance metric $\mathbf{B}$, similar to how PCA constructs a covariance matrix $\mathbf{C}$ from the variance metric. We can compute the eigenvalues of this distance matrix, $\mathbf{B}$

We find the eigenvalues and eigenvectors of $\mathbf{B}$, denoted $\lambda_j$ and $\mathbf{c}_j$, respectively. Next, we decide on a number of reduced dimensions $k$. Then the new dimension $z_j$ of our multidimensional scaling (where $j$ indexes the number of reduced dimensions $k$) is:

$$
\mathbf{z}_j = \sqrt{\lambda_j} \mathbf{c}_j
$$

where $j = 1, \dots, k$ is the index over the number of _reduced_ dimensions.

We can see that under the hood, PCA and MDS are very similar: they both perform an eigenvalue analysis of a matrix formed using a metric (covariance for PCA, Euclidean distances for MDS).

In [ ]:
%matplotlib notebook

# numbers
import numpy as np
import pandas as pd

# stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# plots
import matplotlib.pyplot as plt
import seaborn as sns

# utils
import os, re
from pprint import pprint

## Load the Data

To load the data, we'll follow steps in the PCA notebook, including standardizing and normalizing it.

In [ ]:
## No learning, no testing.
#testing_df = pd.read_csv('data/optdigits/optdigits.tes',header=None)
#X_testing,  y_testing  = testing_df.loc[:,0:63],  testing_df.loc[:,64]

training_df = pd.read_csv('data/optdigits/optdigits.tra',header=None)
X_training, y_training = training_df.loc[:,0:63], training_df.loc[:,64]

In [ ]:
print X_training.shape
print y_training.shape

In [ ]:
def get_normed_mean_cov(X):
    X_std = StandardScaler().fit_transform(X)
    X_mean = np.mean(X_std, axis=0)
    
    ## Automatic:
    #X_cov = np.cov(X_std.T)
    
    # Manual:
    X_cov = (X_std - X_mean).T.dot((X_std - X_mean)) / (X_std.shape[0]-1)
    
    return X_std, X_mean, X_cov

X_std, X_mean, X_cov = get_normed_mean_cov(X_training)

## Multidimensional Scaling: 2 Dimensions

Projecting the full 64-dimensional input to 2 dimensions will lead to a loss of a lot of information - similar to trying to use only 2 principal components. However, it's still useful to do it and visualize it to see what kind of results we'll get, and extend this approach to higher dimensions.

In [ ]:
from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

In [ ]:
n_samples = 300
similarities = euclidean_distances(X_std[:n_samples])

In [ ]:
mds = manifold.MDS(n_components=2, max_iter=1000, eps=1e-3,
                   n_jobs=1)
pos = mds.fit(similarities).embedding_

In [ ]:
def get_cmap(n):
    #colorz = plt.cm.cool
    colorz = plt.get_cmap('Set1')
    return[ colorz(float(i)/n) for i in range(n)]

colorz = get_cmap(10)
colors = [colorz[yy] for yy in y_training]

fig = plt.figure(figsize=(4,4))
plt.scatter(pos[:,0],pos[:,1],c=colors)
f = plt.gcf()
ax = plt.gca()
ax.set_title('what')
plt.show()

In [ ]:
mds2 = manifold.MDS(n_components=2, max_iter=1000, eps=1e-3,
                   dissimilarity="precomputed", n_jobs=1)
pos2 = mds2.fit(similarities).embedding_

In [ ]:
print pos2.shape

In [ ]:
plt.scatter(pos2[:,0],pos2[:,1],c=colors)
plt.show()

## 3D Manifold

Using a 3-component mutlidimensional scaling model and a 3D scatterplot leads to some improvement:

In [ ]:
n_samples = 400
similarities = euclidean_distances(X_std[:n_samples])

In [ ]:
mds3 = manifold.MDS(n_components=4, max_iter=1000, eps=1e-4,
                   n_jobs=1)
pos3 = mds3.fit(similarities).embedding_

In [ ]:
print pos3.shape

In [ ]:
mds4 = manifold.MDS(n_components=3, max_iter=1000, eps=1e-3,
                   dissimilarity="precomputed", n_jobs=1)
pos4 = mds4.fit(similarities).embedding_

In [ ]:
print pos4.shape

In [ ]:
def get_cmap(n):
    #colorz = plt.cm.cool
    colorz = plt.get_cmap('Set1')
    return[ colorz(float(i)/n) for i in range(n)]

colorz = get_cmap(10)
colors = [colorz[yy] for yy in y_training[:n_samples]]

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


fig = plt.figure(figsize=(5,5))
ax1 = fig.add_subplot(111, projection='3d')

ax1.scatter(pos3[:,0],pos3[:,1],pos3[:,2],c=colors)
    
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('z')

plt.show()